## Upload e Download de arquivos usando o ftplib
***

Como dito anteriormente a maioria dos protocolos funciona por cima de uma estrutura baseada em sockets, um desses casos é o FTP (file transfer protocol) que nos permite transferir arquivos entre cliente e servidor através de dois sockets (um para passar comandos e outro para transferir bytes) utilizando port=21.

Para modelar esse protocolo fazemos uso da biblioteca ftplib

Veja um exemplo de como fazer download

Vamos criar um script para baixar e rodar um arquivo pelo FTP. Usa o ftplib, o objeto FTP que utiliza sockets. FTP se basea em dois sockets (um para data, e outro para controle, nas portas 20 e 21) e impôe messagens com formatos específicos, mas o modulo ftplib esconde a maior parte dos detalhes desse protocolo. Mude o endereço usado para o seu site/arquivo.

Você pode olhar no livro 'Programming Python' 4 edição da página 884 a 889 como fazer o upload e download de uma determinada pasta ou árvore de pastas, isto tudo só para mostrar programaticamente como um sistema de computação em núvem funciona.

***

In [5]:
import os # lida com diretorios
from getpass import getpass # usa o input do python sem aparecer os caracteres
from ftplib import FTP # Objeto responsavel por realizar a conexão

In [6]:
# Variávél que determina se devemos usar o modo ativo para FTP
nonpassive = False

In [7]:
# Nome do arquivo a ser baixado
arquivo = 'README'

In [8]:
# Nome do diretório a ser acessado
diretorio = 'debian'

In [9]:
# Nome do site a ser acessado, pode ser um domínio ou endereço de IP
site = 'ftp.debian.org'

In [10]:
# Poderiamos pegar um usuário e senha para conectar no servidor
# usuario = ('lutz', getpass('Senha: '))
# Se não especificar uma senha e usuário ele vai entrar em modo anônimo
usuario = []

In [11]:
# Iniciamos a conexão
print('Conectando...')

Conectando...


In [12]:
# Criamos a conexão com o servidor do tipo FTP
# através do objeto FTP que recebe o endereço
# do servidor como argumento
conexao = FTP(site)

OSError: [Errno 99] Cannot assign requested address

In [ ]:
# Fazemos o login com as informaçoes do usuário
# poderiamos utilizar o login sem nenhum argumento
# e desta forma estariamos entrando no servidor no
# anonimamente
conexao.login(*usuario)

In [ ]:
# Acessamos um determinado diretório
conexao.cwd(diretorio)

In [ ]:
# Podemos listar os conteúdos do diretório corrente
# utilizando o método retrlines e passando para ele
# o comando LIST
print(conexao.retrlines('LIST'))

In [ ]:
# Se devemos acessar os arquivos no modo passivo ou no modo ativo
if nonpassive:
    conexao.set_pasv(False)

In [ ]:
# Iniciamos o donwload do arquivo
# Abrimos um novo arquivo para armazenar as informações dos arquivos que estamos baixando
with open(arquivo, 'wb') as arquivo_local:
    # Armazena informações do arquivo original no arquivo local
    # Colocamos a string 'RETR' que indica o comando de download para o FTP
    # retrbinary indica que estamos baixando o arquivo no modo binário,
    # poderiamos baixar o arquivo usando retrlines que baixa o arquivo em ASCII
    conexao.retrbinary('RETR ' + arquivo, arquivo_local.write, 1024)
                       
    # Fechamos a conexão
    conexao.quit()

In [ ]:
# Perguntamos se o usuário deseja abrir o arquivo,
# se a resposta for sim nós o abrimos
if input('Abre Arquivo?').lower().startswith('s'):
    from PP4E.System.Media.playfile import playfile
    playfile(arquivo)

***

Vamos organizar as funcionalidades para baixar os arquivos utilizando uma única função.

***

In [1]:
from ftplib import FTP
from os.path import exists

In [2]:
def download(arquivo, side, diretorio, usuario=(), verbose=True, refetch=False):
    """
    Obtem um arquivo de um determinado site no diretório selecionado
    :paramn file: nome do arquivo a ser baixado (str)
    :paramn site: domínio do servidor (str)
    :paramn diretorio: Diretório que se localiza o arquivo no servidor (str)
    :paramn user: tupla contendo o usuário e senha, se estiver vazia o login é feito de forma anônima
    :paramn verbose: bool indicando se devem ser impressas mensagens de operações realizadas
    :paramn refetch: bool indicando se caso o arquivo exista se ele deve ser baixado novamente
    """
    
    # Primeiro verificamos se o arquivo existe, se for o caso e nós não estivermos no modo refetch
    # devemos sair da função
    if exists(arquivo) and not refetch:
        if verbose: pring(arquivo, 'já foi baixado')
        return
    else:
        if verbose: print('Downloading', arquivo)
            
    # Abre o arquivo local
    arquivo_local = open(arquivo, 'wb')
    
    # Encapsulamos as operações de download num bloco de try
    # uma vez que é necessário lidar com possíveis erros de conexão
    try:
        # Criar a conexão com o servidor
        conexao = FTP(site)
        # Realizar o login
        conexao.login(*usuario)
        # Mudar o diretorio
        conexao.cwd(diretorio)
        # Baixar o arquivo no modo binário
        conexao.retrbinary('RETR ' + arquivo, arquivo_local.write, 1024)
        # Fechar a conexao
        conexao.quit()
    finally:
        # Fecha o arquivo
        arquivo_local.close()
        
        if verbose: print('Download Concluído!')

***

Além de fazer o download, podemos também enviar um arquivo através do protocolo FTP. Veja a seguir uma maneira de faze-lo.

***

In [4]:
def upload(arquivo, side, diretorio, usuario=(), verbose=True):
    """
    Envia um arquivo para um determinado site no diretório selecionado
    :paramn file: nome do arquivo a ser enviado (str)
    :paramn site: domínio do servidor (str)
    :paramn diretorio: Diretório que se deseja enviar o arquivo no servidor (str)
    :paramn user: tupla contendo o usuário e senha, se estiver vazia o login é feito de forma anônima
    :paramn verbose: bool indicando se devem ser impressas mensagens de operações realizadas
    """
    
    if verbose: print('Enviando', arquivo)
            
    # Abre o arquivo local
    arquivo_local = open(arquivo, 'rb')
    
    # Encapsulamos as operações de envio num bloco de try
    # uma vez que é necessário lidar com possíveis erros de conexão
    try:
        # Criar a conexão com o servidor
        conexao = FTP(site)
        # Realizar o login
        conexao.login(*usuario)
        # Mudar o diretorio
        conexao.cwd(diretorio)
        # Baixar o arquivo no modo binário
        conexao.storbinary('STOR ' + arquivo, arquivo_local, 1024)
        # Fechar a conexao
        conexao.quit()
    except Exception as e:
        print(str(e))
    finally:
        # Fecha o arquivo
        arquivo_local.close()
        
        if verbose: print('Envio Concluído!')